In [ ]:
%%capture
!pip install streamlit
!pip install python-dotenv
!pip install pyodbc
!pip install pymssql sqlalchemy
!pip install google-generativeai
!apt-get install unixodbc-dev
!pip install langchain
!pip install langchain-openai
!pip install huggingface_hub
!pip install pyngrok
!pip install transformers datasets torch
!pip install gradio
!npm install -g localtunnel
!pip install langchain-community
!pip install chromadb

In [ ]:
import os
import pandas as pd
from langchain.chains import RetrievalQA, LLMChain  # LLMChain doğru modülden import ediliyor
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
import torch
from transformers import BertTokenizer, BertModel
import gradio as gr
from huggingface_hub import login
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import transformers
import torch
from huggingface_hub import login
# Hugging Face API Token ile giriş yapın
login(token="")

# Llama modelini başlatma
llm = ChatOpenAI(model="meta-llama/Llama-3.2-3B-Instruct", openai_api_key="")
memory = ConversationBufferWindowMemory(k=3, return_messages=True)
conversation = ConversationChain(memory=memory, llm=llm)


model_id = "meta-llama/Llama-3.2-3B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

<ipython-input-2-40de98875e52>:24: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3, return_messages=True)
<ipython-input-2-40de98875e52>:25: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(memory=memory, llm=llm)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import pandas as pd
import os

# Kipaş ve Çalışan Bilgileri Yükleniyor
calisan_bilgileri_path = '/content/drive/MyDrive/CalısanBilgileri.csv'
kipas_bilgileri_path = '/content/drive/MyDrive/KipasBilgiileri.csv'

calisanlar_df = pd.read_csv(calisan_bilgileri_path)
kipas_bilgileri_df = pd.read_csv(kipas_bilgileri_path)

# BERT Türkçe Modeli ile Embedding
embedding_model = HuggingFaceEmbeddings(model_name="dbmdz/bert-base-turkish-cased")

# Veritabanı dosyasının yolu
db_name = "chroma_db_kipas"
db_path = os.path.join(db_name)

# Veritabanı mevcutsa, yükleyin
if os.path.exists(db_path):
    db = Chroma(persist_directory=db_path, embedding_function=embedding_model)
else:
    # Veritabanı yoksa oluşturun, ve verileri yükleyin
    kipas_metni = " ".join(kipas_bilgileri_df['İçerik'].tolist())  # Kipaş bilgilerini birleştir
      # Çalışanlarla ilgili metinler

    # String verilerini Document nesnelerine dönüştür
    documents = [
        Document(page_content=kipas_metni),

    ]

    # Veritabanı oluşturuluyor (embedding parametresi doğru olarak belirtiliyor)
    db = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=db_path)

# Veritabanı hazırsa, sorguları çalıştırabiliriz
print("Veritabanı başarıyla oluşturuldu ve hazır!")




# Llama Chat Fonksiyonu
def Llama_Chat(system_role, user_msg):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_msg},
    ]
    outputs = pipeline(messages, max_new_tokens=256)

    # Çıktıyı kontrol et
    if outputs and len(outputs) > 0 and "generated_text" in outputs[0]:
        reply = outputs[0]["generated_text"][-1]["content"]
        return reply
    return "Üzgünüm, şu anda yanıt üretemiyorum."

# Chatbot Fonksiyonu
def chatbot(sicil_no, bilgi_turu, user_msg):
    try:
        sicil_no = int(sicil_no)  # Sicil numarasını integer'a çevir
    except ValueError:
        return "⚠ Geçersiz sicil numarası! Lütfen yalnızca rakam girin."

    # Çalışanı bul
    calisan = calisanlar_df[calisanlar_df['Sicil No'] == sicil_no]

    if calisan.empty:
        return "⚠ Sicil numarası sistemde bulunamadı! Lütfen geçerli bir numara girin."

    # Çalışan bilgilerini çek
    ad = calisan.iloc[0]['Ad']
    soyad = calisan.iloc[0]['Soyad']
    pozisyon = calisan.iloc[0]['Pozisyon']
    departman = calisan.iloc[0]['Departman']
    maas = calisan.iloc[0]['Maaş']
    baslama_tarihi = calisan.iloc[0]['İşe Başlama Tarihi']
    yonetici = calisan.iloc[0]['Yönetici']
    email = calisan.iloc[0]['Email']
    telefon = calisan.iloc[0]['Telefon']
    izin_gunleri = calisan.iloc[0]['İzin Günleri']
    hastalik_izni = calisan.iloc[0]['Hastalık İzni']
    performans_deg = calisan.iloc[0]['Performans Değerlendirme']

    system_role = """
    Kipaş Holding'in İK Chatbotusun. Çalışanlarla ilgili sorulara hızlı ve doğru yanıtlar veriyorsun.
    Bilgileri güvenli bir şekilde paylaşıyor ve gerektiğinde detaylı açıklamalar sunuyorsun.
    """

    # Seçilen bilgi türüne göre yanıt üret
    bilgiler = {
        "Maaş": f"💰 Maaşınız: {maas} TL.",
        "İzin Durumu": f"📆 Kalan izin günleriniz: {izin_gunleri} gün.",
        "Departman": f"🏢 {departman} departmanında çalışıyorsunuz.",
        "Pozisyon": f"📌 Pozisyonunuz: {pozisyon}.",
        "Yönetici": f"👨‍💼 Yöneticiniz: {yonetici}.",
        "İşe Başlama Tarihi": f"📅 İşe başlama tarihiniz: {baslama_tarihi}.",
        "Telefon": f"📞 Telefon numaranız: {telefon}.",
        "Email": f"📧 Email adresiniz: {email}.",
        "Performans Değerlendirme": f"📊 Performans değerlendirme puanınız: {performans_deg}.",
        "Hastalık İzni": f"🤒 Hastalık izniniz: {hastalik_izni} gün."
    }

    # Eğer bilgi türü listede varsa, yanıtı döndür
    if bilgi_turu in bilgiler:
        return bilgiler[bilgi_turu]

    # Eğer kullanıcı özel bir soru sormuşsa, Llama modeline gönder
    if user_msg.strip():
        return Llama_Chat(system_role, user_msg)

    return "⚠ Geçerli bir bilgi türü seçiniz veya bir mesaj yazınız."

# Gradio Arayüzü
gui = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(label="📌 Çalışan Sicil Numarası"),
        gr.Dropdown(
            ["Maaş", "İzin Durumu", "Departman", "Pozisyon", "Yönetici", "İşe Başlama Tarihi",
             "Telefon", "Email", "Performans Değerlendirme", "Hastalık İzni"],
            label="📋 Bilgi Türü"
        ),
        gr.Textbox(label="✍ Sorunuzu Buraya Yazın (Opsiyonel)")
    ],
    outputs="text",
    title="💼 İK Chatbot",
    description="📌 Çalışan Sicil Numarasını Girin, Bilgi Seçin veya Mesaj Yazın.",
    live=False
)

# Chatbot'u Başlat
gui.launch(share=True)


Veritabanı başarıyla oluşturuldu ve hazır!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87cb90a161a4625729.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:





# Chatbot Fonksiyonu
def chatbot(sicil_no, bilgi_turu, user_msg):
    try:
        sicil_no = int(sicil_no)  # Sicil numarasını integer'a çevir
    except ValueError:
        return "⚠ Geçersiz sicil numarası! Lütfen yalnızca rakam girin."

    # Çalışanı bul
    calisan = calisanlar_df[calisanlar_df['Sicil No'] == sicil_no]

    if calisan.empty:
        return "⚠ Sicil numarası sistemde bulunamadı! Lütfen geçerli bir numara girin."

    # Çalışan bilgilerini çek
    ad = calisan.iloc[0]['Ad']
    soyad = calisan.iloc[0]['Soyad']
    pozisyon = calisan.iloc[0]['Pozisyon']
    departman = calisan.iloc[0]['Departman']
    maas = calisan.iloc[0]['Maaş']
    baslama_tarihi = calisan.iloc[0]['İşe Başlama Tarihi']
    yonetici = calisan.iloc[0]['Yönetici']
    email = calisan.iloc[0]['Email']
    telefon = calisan.iloc[0]['Telefon']
    izin_gunleri = calisan.iloc[0]['İzin Günleri']
    hastalik_izni = calisan.iloc[0]['Hastalık İzni']
    performans_deg = calisan.iloc[0]['Performans Değerlendirme']

    system_role = """
    Kipaş Holding'in İK Chatbotusun. Çalışanlarla ilgili sorulara hızlı ve doğru yanıtlar veriyorsun.
    Bilgileri güvenli bir şekilde paylaşıyor ve gerektiğinde detaylı açıklamalar sunuyorsun.
    """

    # Seçilen bilgi türüne göre yanıt üret
    bilgiler = {
        "Maaş": f"💰 Maaşınız: {maas} TL.",
        "İzin Durumu": f"📆 Kalan izin günleriniz: {izin_gunleri} gün.",
        "Departman": f"🏢 {departman} departmanında çalışıyorsunuz.",
        "Pozisyon": f"📌 Pozisyonunuz: {pozisyon}.",
        "Yönetici": f"👨‍💼 Yöneticiniz: {yonetici}.",
        "İşe Başlama Tarihi": f"📅 İşe başlama tarihiniz: {baslama_tarihi}.",
        "Telefon": f"📞 Telefon numaranız: {telefon}.",
        "Email": f"📧 Email adresiniz: {email}.",
        "Performans Değerlendirme": f"📊 Performans değerlendirme puanınız: {performans_deg}.",
        "Hastalık İzni": f"🤒 Hastalık izniniz: {hastalik_izni} gün."
    }

    # Eğer bilgi türü listede varsa, yanıtı döndür
    if bilgi_turu in bilgiler:
        return bilgiler[bilgi_turu]

    # Eğer kullanıcı özel bir soru sormuşsa, RAG modeline gönder
    if user_msg.strip():
        return rag_query(user_msg)

    return "⚠ Geçerli bir bilgi türü seçiniz veya bir mesaj yazınız."

# Gradio Arayüzü
gui = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(label="📌 Çalışan Sicil Numarası"),
        gr.Dropdown(
            ["Maaş", "İzin Durumu", "Departman", "Pozisyon", "Yönetici", "İşe Başlama Tarihi",
             "Telefon", "Email", "Performans Değerlendirme", "Hastalık İzni"],
            label="📋 Bilgi Türü"
        ),
        gr.Textbox(label="✍ Sorunuzu Buraya Yazın (Opsiyonel)")
    ],
    outputs="text",
    title="💼 İK Chatbot",
    description="📌 Çalışan Sicil Numarasını Girin, Bilgi Seçin veya Mesaj Yazın.",
    live=False
)

# Chatbot'u Başlat
gui.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://caba2c14abb80d8279.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Çalışan bilgileri dosyasını yükleme
calisanlar_df = pd.read_csv('/content/drive/MyDrive/CalısanBilgileri.csv')

# Önceden tanımlı yanıtları içeren sözlük
yanitlar = {
    "Maaş": "Maaşınız belirtilen miktardadır.",
    "İzin Durumu": "Kalan izin günleriniz belirtilen miktardadır.",
    "Departman": "Çalıştığınız departman belirtilmiştir.",
    "Pozisyon": "Pozisyonunuz belirtilmiştir.",
    "Yönetici": "Yöneticiniz belirtilmiştir.",
    "İşe Başlama Tarihi": "İşe başlama tarihiniz belirtilmiştir.",
    "Telefon": "Telefon numaranız belirtilmiştir.",
    "Email": "Email adresiniz belirtilmiştir.",
    "Performans Değerlendirme": "Performans değerlendirme puanınız belirtilmiştir.",
    "Hastalık İzni": "Hastalık izniniz belirtilmiştir."
}


# Geliştirilmiş Chatbot Promtu
SYSTEM_PROMPT = """
Sen, Kipaş Holding'in resmi İnsan Kaynakları Chatbotusun. Şirket içindeki çalışanlara yardımcı olmak için buradasın.

**Yetkinliklerin:**
- Çalışanların maaş, departman, yöneticileri gibi bilgilerini doğrulayarak yanıt verebilirsin.
- Şirket politikaları hakkında bilgi vererek işten çıkış, izin süreçleri gibi konular hakkında rehberlik edebilirsin.
- Kullanıcılarla dostça ve profesyonel bir üslup kullanarak konuşabilirsin.
- Eğer bir sorunun cevabı kesin olarak verilmiş bir prosedür değilse, bunu analiz ederek en uygun yanıtı üretmelisin.
- Eğer çalışan kişisel bilgilerini sorguluyorsa, doğrudan ilgili bilgiyi vermelisin.

**Şirket Kuralları ve Prosedürler:**
1. Çalışanlar yıllık izin kullanmadan önce yöneticilerinden onay almalıdır.
2. İşten ayrılma sürecinde, çalışanlar en az 2 hafta önceden İK departmanına bildirim yapmalıdır.
3. Çalışanlar, maaş bordrosu almak için İK sistemine giriş yapmalıdır.
4. Hastalık izni almak için doktor raporu gereklidir.

**Örnek Yanıtlar:**
- Kullanıcı: "Maaşım ne kadar?"
  Yanıt: "Maaşınız 25.000 TL'dir."

- Kullanıcı: "İşten çıkış süreci nasıl işliyor?"
  Yanıt: "İşten ayrılmak için en az 2 hafta önceden İnsan Kaynakları departmanına haber vermelisiniz."

**Serbest Yanıtlar İçin:**
Eğer kullanıcı yukarıdaki konuların dışında bir şey sorarsa, konuyu analiz et ve uygun bir yanıt üret.
"""

In [ ]:
# Çalışan verilerini yükle
try:
    calisanlar_df = pd.read_csv('/content/drive/MyDrive/CalısanBilgileri.csv')
except FileNotFoundError:
    raise FileNotFoundError("Çalışan verisi bulunamadı! CSV dosyanızı kontrol edin.")

# Llama Chat Fonksiyonu
def Llama_Chat(system_role, user_msg):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_msg},
    ]
    outputs = pipeline(messages, max_new_tokens=256)

    # Çıktıyı kontrol et
    if outputs and len(outputs) > 0 and "generated_text" in outputs[0]:
        reply = outputs[0]["generated_text"][-1]["content"]
        return reply
    return "Üzgünüm, şu anda yanıt üretemiyorum."

# Chatbot Fonksiyonu
def chatbot(sicil_no, bilgi_turu, user_msg):
    try:
        sicil_no = int(sicil_no)  # Sicil numarasını integer'a çevir
    except ValueError:
        return "⚠ Geçersiz sicil numarası! Lütfen yalnızca rakam girin."

    # Çalışanı bul
    calisan = calisanlar_df[calisanlar_df['Sicil No'] == sicil_no]

    if calisan.empty:
        return "⚠ Sicil numarası sistemde bulunamadı! Lütfen geçerli bir numara girin."

    # Çalışan bilgilerini çek
    ad = calisan.iloc[0]['Ad']
    soyad = calisan.iloc[0]['Soyad']
    pozisyon = calisan.iloc[0]['Pozisyon']
    departman = calisan.iloc[0]['Departman']
    maas = calisan.iloc[0]['Maaş']
    baslama_tarihi = calisan.iloc[0]['İşe Başlama Tarihi']
    yonetici = calisan.iloc[0]['Yönetici']
    email = calisan.iloc[0]['Email']
    telefon = calisan.iloc[0]['Telefon']
    izin_gunleri = calisan.iloc[0]['İzin Günleri']
    hastalik_izni = calisan.iloc[0]['Hastalık İzni']
    performans_deg = calisan.iloc[0]['Performans Değerlendirme']

    system_role = """
    Kipaş Holding'in İK Chatbotusun. Çalışanlarla ilgili sorulara hızlı ve doğru yanıtlar veriyorsun.
    Bilgileri güvenli bir şekilde paylaşıyor ve gerektiğinde detaylı açıklamalar sunuyorsun.
    """

    # Seçilen bilgi türüne göre yanıt üret
    bilgiler = {
        "Maaş": f"💰 Maaşınız: {maas} TL.",
        "İzin Durumu": f"📆 Kalan izin günleriniz: {izin_gunleri} gün.",
        "Departman": f"🏢 {departman} departmanında çalışıyorsunuz.",
        "Pozisyon": f"📌 Pozisyonunuz: {pozisyon}.",
        "Yönetici": f"👨‍💼 Yöneticiniz: {yonetici}.",
        "İşe Başlama Tarihi": f"📅 İşe başlama tarihiniz: {baslama_tarihi}.",
        "Telefon": f"📞 Telefon numaranız: {telefon}.",
        "Email": f"📧 Email adresiniz: {email}.",
        "Performans Değerlendirme": f"📊 Performans değerlendirme puanınız: {performans_deg}.",
        "Hastalık İzni": f"🤒 Hastalık izniniz: {hastalik_izni} gün."
    }

    # Eğer bilgi türü listede varsa, yanıtı döndür
    if bilgi_turu in bilgiler:
        return bilgiler[bilgi_turu]

    # Eğer kullanıcı özel bir soru sormuşsa, Llama modeline gönder
    if user_msg.strip():
        return Llama_Chat(system_role, user_msg)

    return "⚠ Geçerli bir bilgi türü seçiniz veya bir mesaj yazınız."

# Gradio Arayüzü
gui = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(label="📌 Çalışan Sicil Numarası"),
        gr.Dropdown(
            ["Maaş", "İzin Durumu", "Departman", "Pozisyon", "Yönetici", "İşe Başlama Tarihi",
             "Telefon", "Email", "Performans Değerlendirme", "Hastalık İzni"],
            label="📋 Bilgi Türü"
        ),
        gr.Textbox(label="✍ Sorunuzu Buraya Yazın (Opsiyonel)")
    ],
    outputs="text",
    title="💼 İK Chatbot",
    description="📌 Çalışan Sicil Numarasını Girin, Bilgi Seçin veya Mesaj Yazın.",
    live=False
)

# Chatbot'u Başlat
gui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://044fe3f5652218a098.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
